<a href="https://colab.research.google.com/github/Jackson2706/Machine-Translation/blob/main/src/Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
%cd drive/MyDrive/Pet-Project/Machine-Translation

[Errno 2] No such file or directory: 'drive/MyDrive/Pet-Project/Machine-Translation'
/content/drive/MyDrive/Pet-Project/Machine-Translation


# 1. Config

In [39]:
config = {
    "Train_dataset_folder_path": "data/train-en-vi"
}

#2. Dataset

In [40]:
# import libraries
!pip install langid
!pip install torchtext
import os
import re
import string
from torchtext.vocab import build_vocab_from_iterator
from  torchtext.data.utils import get_tokenizer
import torch
from torch.utils.data import Dataset
from langid.langid import LanguageIdentifier, model


## 2.1 - Preprocess data

In [41]:
# Loại bỏ các mẫu có chứa kí tư không phải là tiếng anh
def identify_lang(en_data, vi_data, threshold: float = 0.9):
  identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
  en_list = []
  vi_list = []
  for en_sentence, vi_sentence in zip(en_data, vi_data):
    en_score = identifier.classify(en_sentence)
    vi_score = identifier.classify(vi_sentence)
    if (en_score[0] == 'en' and en_score[1] >= threshold) and (vi_score[0] == 'vi' and vi_score[1] >= threshold):
      en_list.append(en_sentence)
      vi_list.append(vi_sentence)
  return en_list, vi_list

In [50]:
# Tiền xử lý data
def preprocessing_text(text: str):
    """
    Preprocesses text by removing special patterns, punctuation, digits, and emojis.

    Args:
        text (str): The input text to be preprocessed.

    Returns:
        str: Clean text containing only Vietnamese characters.
    """
    # Define the URL pattern
    url_pattern = re.compile(r"https?://\s+\www\.\s+")
    # Replace URLs with whitespace
    text = url_pattern.sub(r" ", text)

    # Define the HTML pattern
    html_pattern = re.compile(r"<[^<>]+>")
    # Replace HTML patterns with whitespace
    text = html_pattern.sub(" ", text)

    # Define punctuation and digits pattern
    replace_chars = list(string.punctuation + string.digits)
    # Replace punctuation and digits with whitespace
    for char in replace_chars:
        text = text.replace(char, " ")

    # Define the emoji pattern
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U0001F1F2-\U0001F1F4"  # Macau flag
        "\U0001F1E6-\U0001F1FF"  # flags
        "\U0001F600-\U0001F64F"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U0001F1F2"
        "\U0001F1F4"
        "\U0001F620"
        "\u200d"
        "\u2640-\u2642"
        "]+",
        flags=re.UNICODE,
    )
    # Replace emojis with whitespace
    text = emoji_pattern.sub(r" ", text)

    # Remove duplicate whitespace
    text = re.sub(r'\s+', ' ', text)

    # Return lowercase text
    return text.lower()

def preprocess(data_list):
  process_data_list = []
  for sentence in data_list:
    process_data_list.append(preprocessing_text(sentence))
  return process_data_list

## 2.2 - Tokenizer and create vocab

In [43]:
def yield_tokens(sentences, tokenizer):
  for sentence in sentences:
    yield tokenizer(sentence)

In [44]:

def build_vocabulary(sentences, tokenizer):
  vocabulary = build_vocab_from_iterator(yield_tokens(sentences, tokenizer), specials=["unk"])
  vocabulary.set_default_index(vocabulary["unk"])
  return vocabulary

## 2.3 - Test

In [45]:
en_list = []
with open("data/train-en-vi/train.en") as file:
  for sentence in file:
    en_list.append(sentence)

vi_list = []
with open("data/train-en-vi/train.vi") as file:
  for sentence in file:
    vi_list.append(sentence)



In [46]:
clean_en_list, clean_vi_list = identify_lang(en_list, vi_list, 0.9)
print(f"Dữ liệu data ban đầu: {len(en_list)} - {len(vi_list)}")
print(f"Dũ liệu data sau khi xủ lý: {len(clean_en_list)} - {len(clean_vi_list)}")

Dữ liệu data ban đầu: 133317 - 133317
Dũ liệu data sau khi xủ lý: 128999 - 128999


In [51]:
print(f"{clean_en_list[56]} - {clean_vi_list[56]}")

clean_en_data = preprocess(clean_en_list)
clean_vi_data = preprocess(clean_vi_list)
print(f"{clean_en_data[56]} - {clean_vi_data[56]}")


You can program the hundreds of muscles in your arm .
 - Bạn có thể lập trình cho hàng trăm cơ bắp trong cánh tay .

you can program the hundreds of muscles in your arm  - bạn có thể lập trình cho hàng trăm cơ bắp trong cánh tay 


In [52]:
en_tokenizer = get_tokenizer('basic_english')
en_vocab = build_vocabulary(clean_en_data, en_tokenizer)
print(f"English vocab: {en_vocab.get_stoi()}")

vi_tokenizer = get_tokenizer('basic_english')
vi_vocab = build_vocabulary(clean_vi_data, vi_tokenizer)
print(f"Vietnamese vocab: {vi_vocab.get_stoi()}")


English vocab: {'ālep': 40956, 'über': 40955, 'élysées': 40954, 'zz': 40947, 'zywiec': 40946, 'zyprexa': 40945, 'zworykin': 40943, 'zweep': 40941, 'zumthor': 40940, 'zuccotti': 40939, 'zorba': 40935, 'zopa': 40934, 'zoophilia': 40933, 'zoilo': 40926, 'zira': 40924, 'zimmer': 40922, 'zhuzhou': 40919, 'zhou': 40918, 'zhong': 40917, 'zheng': 40916, 'zheimers': 40915, 'zeroed': 40914, 'zeppelins': 40913, 'zenola': 40912, 'zem': 40909, 'zealotry': 40906, 'zealander': 40905, 'zavorotnya': 40903, 'zapping': 40901, 'zapalu': 40899, 'zanzibar': 40898, 'zany': 40897, 'zagat': 40894, 'zackie': 40892, 'yug': 40891, 'yquem': 40890, 'ypo': 40889, 'youtubes': 40887, 'yourselfers': 40884, 'youporn': 40882, 'youku': 40879, 'yor': 40875, 'yoon': 40874, 'yoohoo': 40873, 'yoked': 40869, 'yoke': 40868, 'yiddish': 40860, 'yeses': 40856, 'yen': 40854, 'yellowcake': 40850, 'yeasayer': 40848, 'yearnings': 40847, 'yearbooks': 40846, 'yau': 40843, 'yates': 40842, 'yarkoni': 40838, 'yankee': 40836, 'yams': 40834,

## 2.4 - Create dataset

In [56]:
class EnglishVietNamDataset(Dataset):
  def __init__(self, data_folder, phase, en_tokenizer, vi_tokenizer ,max_sequence_length, threshold, en2vi = True):
    en_data_file_path = os.path.join(data_folder, f"{phase}.en")
    vi_data_file_path = os.path.join(data_folder, f"{phase}.vi")

    raw_en_data = []
    raw_vi_data = []
    with open(en_data_file_path, "r") as file:
      for line in file:
        raw_en_data.append(line)

    with open(vi_data_file_path, "r") as file:
      for line in file:
        raw_vi_data.append(line)
    clean_en_data, clean_vi_data = identify_lang(raw_en_data, raw_vi_data, threshold)
    self.input_en_data = preprocess(clean_en_data)
    self.input_vi_data = preprocess(clean_vi_data)
    self.en_vocab = build_vocabulary(self.input_en_data, en_tokenizer)
    self.vi_vocab = build_vocabulary(self.input_vi_data, vi_tokenizer)
    self.en_tokenizer = en_tokenizer
    self.vi_tokenizer = vi_tokenizer
    self.max_sequence_length = max_sequence_length
    self.en2vi = en2vi

  def __len__(self):
    return len(self.raw_en_data)

  def __getitem__(self, index):
    en_data = self.input_en_data[index]
    vi_data = self.input_vi_data[index]
    if self.en2vi:
      en_vectorize_data = self._vectorize(en_data, self.en_tokenizer, self.en_vocab, self.max_sequence_length)
      vi_vectorize_data = self._vectorize(vi_data, self.vi_tokenizer, self.vi_vocab, self.max_sequence_length, True)
      en_tensor = torch.tensor(en_vectorize_data, dtype=torch.long)
      vi_tensor = torch.tensor(vi_vectorize_data, dtype=torch.long)
      return en_tensor, vi_tensor
    else:
      en_vectorize_data = self._vectorize(en_data, self.en_tokenizer, self.en_vocab, self.max_sequence_length, True)
      vi_vectorize_data = self._vectorize(vi_data, self.vi_tokenizer, self.vi_vocab, self.max_sequence_length)
      en_tensor = torch.tensor(en_vectorize_data, dtype=torch.long)
      vi_tensor = torch.tensor(vi_vectorize_data, dtype=torch.long)
      return en_tensor, vi_tensor

  def _vectorize(self, text, tokenizer, vocab, max_sequence_length, add_sos = True):
    tokens = tokenizer(text)
    tokens = [vocab[token] for token in tokens] + [vocab["<eos>"]]
    if add_sos:
      tokens = [vocab["<sos>"]] + tokens
    token_ids = tokens[:max_sequence_length] + [vocab["<pad>"]] * max((max_sequence_length - len(tokens)), 0)
    return token_ids



In [ ]:
# Test

en_tokenizer = get_tokenizer('basic_english')
vi_tokenizer = get_tokenizer('basic_english')

train_dataset = EnglishVietNamDataset(data_folder="data/train-en-vi",
                                      phase="train",
                                      en_tokenizer=en_tokenizer,
                                      vi_tokenizer=vi_tokenizer,
                                      max_sequence_length=50,
                                      threshold=0.95,
                                      en2vi=True)

print(train_dataset[6])